# Export onnx with your trained weights

In [1]:
import os
def get_latest_checkpoint(base_dir):
    ckpt_file = os.path.join(base_dir, "last_checkpoint")
    if os.path.exists(ckpt_file):
        with open(ckpt_file, "r") as f:
            relative_ckpt_path = f.readline().strip()
            full_ckpt_path = os.path.join(base_dir, relative_ckpt_path)
            if os.path.exists(full_ckpt_path):
                return full_ckpt_path
    return None

# the logs stored in work_dirs/resnet101_8xb32_planenbike/
latest_ckpt = get_latest_checkpoint("mmpretrain/work_dirs/resnet101_8xb32_planenbike/")
print(f"Using checkpoint: {latest_ckpt}")

Using checkpoint: /home/test/carasml/classification/mm_Resnet/mmpretrain/work_dirs/resnet101_8xb32_planenbike/epoch_100.pth


In [2]:
# revise the code according to your model
# the second line is the location of exporting script, base on your nature of your task (classification/ detection/ segmentation)
# the third line is the location of configs of your model
# the fourth line is the location of weights (pth) of your model
# the fifth line is a sample of image
# the sixth line is exporting location
# the seventh line is choosing model to run on cpu or cuda

In [3]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/resnet101_8xb32_planenbike/resnet101_8xb32_planenbike.py \
{latest_ckpt} \
mmpretrain/caltech101/val/airplane/image_0800.jpg \
--work-dir mmdeploy_model/resnet \
--device cpu

07/09 13:39:39 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
07/09 13:39:40 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:39:40 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
Loads checkpoint by local backend from path: /home/test/carasml/classification/mm_Resnet/mmpretrain/work_dirs/resnet101_8xb32_planenbike/e

⚠️ **Note on Running Deployment Cells**

Due to how Python's `multiprocessing` module interacts with interactive environments like Jupyter Notebook, running multiple `%run mmdeploy/tools/deploy.py` commands consecutively **without restarting the kernel** may lead to the following error:

" AttributeError: module 'main' has no attribute 'spec' "


This is a known limitation when using `spawn` multiprocessing context in scripts executed via `%run`.

🛠 **Recommended Action**:  
After executing a deployment script, please **restart the Jupyter kernel** before running another `%run mmdeploy/tools/deploy.py ...` command. You can do this by clicking:

> Kernel → Restart Kernel

This will prevent the multiprocessing error and ensure stable behavior.


In [1]:
import os
def get_latest_checkpoint(base_dir):
    ckpt_file = os.path.join(base_dir, "last_checkpoint")
    if os.path.exists(ckpt_file):
        with open(ckpt_file, "r") as f:
            relative_ckpt_path = f.readline().strip()
            full_ckpt_path = os.path.join(base_dir, relative_ckpt_path)
            if os.path.exists(full_ckpt_path):
                return full_ckpt_path
    return None

# the logs stored in work_dirs/resnet101_8xb32_planenbike/
latest_ckpt = get_latest_checkpoint("mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px_VOC/")
print(f"Using checkpoint: {latest_ckpt}")

Using checkpoint: /home/test/carasml/classification/mm_Resnet/mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px_VOC/epoch_20.pth


In [2]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px_VOC/resnet101-csra_1xb16_voc07-448px_VOC.py \
{latest_ckpt} \
mmpretrain/VOC2007/JPEGImages/000001.jpg \
--work-dir mmdeploy_model/csra \
--device cpu

07/09 13:57:34 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
07/09 13:57:35 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:57:35 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
Loads checkpoint by local backend from path: /home/test/carasml/classification/mm_Resnet/mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-4

# Inference the onnx

In [3]:
# providers set provider priority cuda or cpu
import onnxruntime
sess = onnxruntime.InferenceSession("mmdeploy_model/resnet/end2end.onnx", providers=["CUDAExecutionProvider"])
print(sess.get_providers())

['CUDAExecutionProvider', 'CPUExecutionProvider']


In [5]:
'''
from mmdeploy.apis import inference_model
is the official method to run an onnx, but it require torch
'''
#Classification
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/resnet101_8xb32_planenbike/resnet101_8xb32_planenbike.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/resnet/end2end.onnx'],
    img='mmpretrain/caltech101/val/airplane/image_0800.jpg',
    device='cpu')
print(result)

07/09 13:40:21 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:40:21 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:40:21 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "backend_classifiers" registry tree. As a workaround, the current "backend_classifiers" registry in "mmdeploy" is used to build instance. This ma

In [6]:
'''
this is an unofficial method to run an onnx that require no torch
however, you need to check the 'model_cfg' to understand how the image was preprocessed
which include normalization method and resize and padding method
you may search keywords "data_preprocessor" and "mean" and "std" for normalization 
and "pipeline" and "scale" for resize and padding method
'''
#Classification 
import onnxruntime
import numpy as np
from PIL import Image

# we resize with longer edge == target size, keep ratio, and pad on short edge
def resize_and_pad(image, target_size, pad_color=(114, 114, 114)):
    # Get original size
    w, h = image.size

    # Determine scale factor and new size
    scale = target_size / max(w, h)
    new_w, new_h = int(w * scale), int(h * scale)

    # Resize the image with aspect ratio preserved
    image = image.resize((new_w, new_h), Image.BILINEAR)

    # Create a new square image and paste the resized image in the center
    new_image = Image.new("RGB", (target_size, target_size), pad_color)
    paste_x = (target_size - new_w) // 2
    paste_y = (target_size - new_h) // 2
    new_image.paste(image, (paste_x, paste_y))

    return new_image
    
# Load the ONNX model with cpu or cuda execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/resnet/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/caltech101/val/airplane/image_0800.jpg"

# Load and preprocess the image
image = Image.open(image_path)
image = resize_and_pad(image, target_size=224)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.transpose(image, (2,0,1))
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            125.307,
            122.961,
            113.8575,
        ],
        std=[
            51.5865,
            50.847,
            51.255,
        ],
        to_rgb=False)
'''
#as to_rgb=False, we swap the red and blue channel
input_array = input_array[:,[2, 1, 0], :, :]
#normalize image
mean = np.array([125.307, 122.961, 113.8575]).reshape(1, 3, 1, 1) 
std = np.array([51.5865, 50.847, 51.255]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

# Print the output
print("Model output:", output)


Available providers: ['CPUExecutionProvider']
Model output: [[9.9952507e-01 4.7489296e-04]]


In [4]:
#Multi-label Classification 
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px_VOC/resnet101-csra_1xb16_voc07-448px_VOC.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/csra/end2end.onnx'],
    img='mmpretrain/VOC2007/JPEGImages/000001.jpg',
    device='cpu')
print(result)

07/09 13:58:26 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:58:26 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:58:26 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "backend_classifiers" registry tree. As a workaround, the current "backend_classifiers" registry in "mmdeploy" is used to build instance. This ma

In [5]:
#Multi-label Classification without torch (may have little difference with official)
import onnxruntime
import numpy as np
from PIL import Image
import cv2
'''
according to config
    dataset=dict(
        data_root='data/VOC2007',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(scale=448, type='Resize'),
'''
# we resize
def resize(image, target_size):
    # Resize the image
    image = image.resize((target_size, target_size), Image.BILINEAR)

    return image
    
# Load the ONNX model with CUDA execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/csra/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/VOC2007/JPEGImages/000001.jpg"

# Load and preprocess the image
image = Image.open(image_path)
image = resize(image, target_size=448)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.transpose(image, (2,0,1))
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            0,
            0,
            0,
        ],
        num_classes=20,
        std=[
            255,
            255,
            255,
        ],
        to_onehot=True,
        to_rgb=True)
'''
#normalize image
mean = np.array([0., 0., 0.]).reshape(1, 3, 1, 1) 
std = np.array([255., 255., 255.]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

threshold = 0.5
# Print the output
print("Model output:", output)
print("Positive class:", np.where(output[0]>threshold)[0])

Available providers: ['CPUExecutionProvider']
Model output: [[7.3173642e-04 6.0661137e-03 3.1151474e-03 2.0319521e-03 1.5517503e-02
  3.7080646e-03 4.8745573e-03 4.3576360e-03 4.5312017e-02 2.1176934e-03
  7.1195364e-03 9.8515677e-01 2.6083291e-03 1.7824173e-03 9.9485207e-01
  1.2965769e-02 7.6399744e-03 3.7139058e-02 7.3951483e-04 1.2001842e-02]]
Positive class: [11 14]
